# Projeto CENAD - Abastecimento de água

Para problemas de otimização, uma busca-se minimizar uma função objetivo $f(x)$,
\begin{equation}
  \boldsymbol{f}^{\boldsymbol{T}} \cdot \boldsymbol{x}
\end{equation}
sujeito a
\begin{equation}
\boldsymbol{A} \boldsymbol{x} \leq \boldsymbol{b},\  \forall i : x_i \geq 0
\end{equation}
sendo que $\boldsymbol{x} = [x_1, x_2, ..., x_n]$ são as variáveis do problema, $\boldsymbol{f} = [c_1, c_2, ..., c_n]$ são os coeficientes da função objetivo, $\boldsymbol{A}$ é uma matriz $m \times n$ e $\boldsymbol{b} = [b_1, b_2, ..., b_m]$ com $b_j \geq 0$.

Para determinar o problema, é necessário definir os coeficientes da função objetivo ($\boldsymbol{f}$) as matrizes de igualdade ($\boldsymbol{A_{eq}}$), desigualdade ($\boldsymbol{A}$), os vetores de igualdade ($\boldsymbol{b_{eq}}$) e desigualdade ($\boldsymbol{b}$). Uma vez determinados, o método Simplex será utilizado para encontrar os coeficientes da função objetivo $\boldsymbol{f}(\boldsymbol{x})$.

No caso do abastecimento, em específico, as matrizes de igualdade é dada pela demanda, as matrizes de desigualdade, pela oferta. A função objetivo descreve a relação município/manancial e a distância $\boldsymbol{x}$ será minimizada.


Para determinar o problema, é necessário definir os coeficientes da função objetivo ($f$) as matrizes de igualdade ($A_{eq}$), desigualdade ($A$), os vetores de igualdade ($b_{eq}$) e desigualdade ($b$).

In [1]:
# Inclua as bibliotecas para ler Excel e Programação Linear
library(Rglpk)
library(readxl)
library(Matrix)
library(optimbase)
library(tibble)

Loading required package: slam
Using the GLPK callable library version 4.65


In [2]:
distancias <- read.delim("../dados/exemplo_distancias.txt", header=FALSE)
oferta <- read.delim("../dados/exemplo_oferta.txt", header=FALSE)
demanda <- read.delim("../dados/exemplo_demanda.txt", header=FALSE)

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on '../dados/exemplo_distancias.txt'”Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on '../dados/exemplo_oferta.txt'”Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on '../dados/exemplo_demanda.txt'”

In [3]:
# Precisamos alocar as matrizes com os tamanhos da oferta e demanda
n_ofer <- nrow(oferta)
n_dema <- nrow(demanda)

In [5]:
distancias

V1,V2,V3
<int>,<int>,<int>
1,10,2
1,10,2
1,10,2
1,10,2
1,10,2


Agora vamos construir as matrizes do problema. A primeira matriz será a da função $f(x)$, que consiste nos coeficientes da função linear $f x$, $f = [c_1 c_2 ... c_n]$ e $x = [x_1 x_2 ... x_n]$

In [6]:
# A função f é uma matriz de uma linha e n_oferta * n_demanda colunas
funcao_f <- matrix(nrow = 1 , ncol =  n_ofer*n_dema)
for (i in 1:n_dema){
  for (j in 1:n_ofer){
    funcao_f[(j+(i-1)*n_ofer)] <- distancias[i,j]
      #*demanda[i,1]
  }
}
funcao_f[is.na(funcao_f)] <- 0 # quando nan (Not A Number), substituir por zero
obj <- funcao_f # Definimos a função objetivo como f

In [7]:
funcao_f

1,10,2,1,10,2,1,10,2,1,10,2,1,10,2


Agora vamos montar a matriz $\boldsymbol{A}$ que relaciona a desigualdade $\boldsymbol{A} \boldsymbol{x} \lt \boldsymbol{b}$, em que $b$ é o vetor que indica a oferta dos mananciais.

In [8]:
funcao_A <- matrix(nrow = n_ofer, ncol = n_dema*n_ofer)
for (i in 1:n_ofer){
  for (j in 1:n_dema){        
      # Cada linha recebe os valores da demanda
    funcao_A[i,i+(j-1)*n_ofer] <- 1
      #as.matrix(demanda[j,1]) 
  }
}
funcao_A[is.na(funcao_A)] <- 0

In [9]:
funcao_A

1,0,0,1,0,0,1,0,0,1,0,0,1,0,0
0,1,0,0,1,0,0,1,0,0,1,0,0,1,0
0,0,1,0,0,1,0,0,1,0,0,1,0,0,1


As restrições do problema são descritas por valores que devem ser iguais a demanda, isto é, os municípios devem receber a totalidade de sua demanda de água (de um único manancial), e nunca menos que sua demanda. E valores que podem ser iguais ou menores, aqueles da oferta dos mananciais, que podem oferecer uma quantidade de água determinada e não mais, mas não existe necessidade de utilizar toda a oferta de cada manancial. Dessa forma, serão construídas duas matrizes $\boldsymbol{A}$, que relaciona a oferta de cada manancial para cada município, e a matriz $\boldsymbol{A_{eq}}$, que relaciona a oferta e a demanda de cada município.

In [10]:
funcao_Aeq <- matrix(nrow = n_dema, ncol = n_dema*n_ofer)
for (i in 1:n_dema){
    for (j in 1:n_ofer){
        # A função Aeq será sempre um ou zero
        funcao_Aeq[i,j+(i-1)*n_ofer] <- 1 
    }
}
funcao_Aeq[is.na(funcao_Aeq)] <- 0
#depois disso, juntar com A
funcao_A_Aeq <- rbind(funcao_A, funcao_Aeq) # Junta-se as duas matriz de igualdade e desigualdade
mat <- funcao_A_Aeq

In [11]:
funcao_Aeq

1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,1,1,1,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,1,1,1,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,1,1,1,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,1,1,1


Agora vamos montar o vetor $\boldsymbol{b}$, que recebe a oferta correspondente as municípios descritos por $\boldsymbol{A}$:

In [12]:
funcao_b <- matrix(nrow = 36, ncol = 1)
funcao_b <- oferta
funcao_b_final <- t(funcao_b)
funcao_b_final[is.na(funcao_b_final)] <- 0

In [13]:
funcao_b

V1
<int>
5
10
10


O vetor $\boldsymbol{b_{eq}}$ é sempre igual a um ou zero e é do tamanho da demanda. Ele define quanto cada município deve receber de água de cada manancial para garantir que receba não menos que sua demanda.

In [14]:
funcao_beq <- matrix(nrow = n_dema, ncol = 1)
for (i in 1:n_dema){
    funcao_beq[i,1] <- as.matrix(demanda[i,1])
    #1
}
funcao_beq_final <- t(funcao_beq)
funcao_beq_final[is.na(funcao_beq_final)] <- 0

#depois disso, juntar com beq
funcao_b_beq <- cbind(funcao_b_final, funcao_beq_final)
rhs <- funcao_b_beq

In [15]:
funcao_beq

1
1
1
2
3


In [16]:
# Lower bound, limite inferior de zero e upper bound, limite superior igual a um
funcao_lb <- matrix(nrow = 1, ncol = n_dema*n_ofer) #matriz em linha só de zeros 
for (i in 1:n_ofer*n_dema){
    funcao_lb[i] <- 0
}                                        #funcao_lb_final <- t(funcao_lb)
funcao_lb[is.na(funcao_lb)] <- 0

funcao_ub <- matrix(nrow = 1, ncol = n_dema*n_ofer) #matriz em linha com numeros quaisquer
for (i in 1:n_ofer*n_dema){
    funcao_ub[i] <- 1
}
#depois disso, transpor
#funcao_ub_final <- t(funcao_ub)
funcao_ub[is.na(funcao_ub)] <- 1
bounds <- list(lower = funcao_lb, upper = funcao_ub )

Enfim, é necessrio definir quais valores da matriz $\boldsymbol{A}$ são de igualdade e desigualdade. Os primeiros 36 valores, ou número de oferta valores, são de desigualdade, os 4599, ou número de demanda valores são de igualdade.

In [17]:
dir <- c(rep('<=', n_ofer),rep('==', n_dema))

O resultado da otimização é salvo em um objeto chamado $solution$. Agora é possível verificar a eficiência em comparação com a distribuição realizada no período estudado.

In [18]:
library(Rglpk)
solution <- Rglpk_solve_LP(obj, mat, dir, rhs, bounds , max = FALSE)

Warning message in any(x[[1L]]):
“coercing argument of type 'double' to logical”Warning message in any(x[[1L]]):
“coercing argument of type 'double' to logical”

In [19]:
solution

$optimum
[1] 11

$solution
 [1] 1 0 0 1 0 0 1 0 0 2 0 0 0 0 3

$status
[1] 0

$solution_dual
 [1] 0 8 0 0 8 0 0 8 0 0 8 0 0 8 0

$auxiliary
$auxiliary$primal
[1] 5 0 3 1 1 1 2 3

$auxiliary$dual
[1] -1  0  0  2  2  2  2  2


$sensitivity_report
[1] NA

In [20]:
val <- solution[['solution']]
iter=1
for (i in 1:n_dema){
    for (j in 1:n_ofer){
        if (val[iter] > 0)
        print(paste0('destino: ',i,', origem: ',j,', carros: ',val[iter]))
        iter=iter+1
    }
}


[1] "destino: 1, origem: 1, carros: 1"
[1] "destino: 2, origem: 1, carros: 1"
[1] "destino: 3, origem: 1, carros: 1"
[1] "destino: 4, origem: 1, carros: 2"
[1] "destino: 5, origem: 3, carros: 3"
